<a href="https://colab.research.google.com/github/patrickdiorio/Spark-Pyspark-Colab-SQL-CopadoMundo/blob/main/ptk_Academia_pyspark_CopaDoMundo_20220321.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Configuração ambiente***

In [1]:
# Instalacao pyspark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 66.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5083fd8af555931516c29f6735a2442487a891067677d9c30349cabfa606a3c8
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
# Iniciar sessão pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
# Versão spark
spark

In [4]:
# Versão python
!python --version

Python 3.7.13


In [5]:
# Bibliotecas necessárias
from pyspark.sql import *
from pandas import *
from google.colab import files
from pyspark import SparkFiles

# ***Copa do Mundo .csv***

In [6]:
# Leitura dos arquivos .csv localizados nas url's abaixo
url1 = 'https://raw.githubusercontent.com/aureliolucassilva/spark/main/WorldCupMatches.csv'
url2 = 'https://raw.githubusercontent.com/aureliolucassilva/spark/main/WorldCupPlayers.csv'
url3 = 'https://raw.githubusercontent.com/aureliolucassilva/spark/main/WorldCups.csv'


# Inicializar dataframes
spark.sparkContext.addFile(url1)
spark.sparkContext.addFile(url2)
spark.sparkContext.addFile(url3)
jogosCopa = spark.read.csv(SparkFiles.get("WorldCupMatches.csv"), inferSchema=True, header=True)
jogadoresCopa = spark.read.csv(SparkFiles.get("WorldCupPlayers.csv"), inferSchema=True, header=True)
copasMundo = spark.read.csv(SparkFiles.get("WorldCups.csv"), inferSchema=True, header=True)

Py4JJavaError: ignored

In [ ]:
# Jogos da copa (Amostra de 10 registros)
jogosCopa.show(10)

In [ ]:
# Jogadores da copa (Amostra de 10 registros)
jogadoresCopa.show(10)

In [ ]:
# Copas do mundo (Amostra de 10 registros)
copasMundo.show(10)

In [ ]:
# Schema do dataframe jogosCopa
jogosCopa.printSchema()

In [ ]:
# Schema do dataframe jogadoresCopa
jogadoresCopa.printSchema()

In [ ]:
# Schema do dataframe copasMundo
copasMundo.printSchema()

# ***Público nos jogos***

In [ ]:
# Maior, menor e média de público nos jogos 
maiorAno = jogosCopa.agg({"Year": "max"}).collect()[0][0]
menorAno = jogosCopa.agg({"Year": "min"}).collect()[0][0]
maiorPublico = jogosCopa.agg({"Attendance": "max"}).collect()[0][0]
menorPublico = jogosCopa.agg({"Attendance": "min"}).collect()[0][0]
mediaPublico = jogosCopa.agg({"Attendance": "avg"}).collect()[0][0]
msg = 'Público entre os anos ' + str(menorAno) + ' e ' + str(maiorAno) + '\n' + \
      'Maior público: ' + str(maiorPublico) + '\n' + \
      'Menor público: ' + str(menorPublico) + '\n' + \
      'Média público: ' + str(mediaPublico) + '\n' 
print(msg)

In [ ]:
# Jogo com maior público
jogosIconicos = jogosCopa.filter(jogosCopa.Attendance == maiorPublico)
jogosIconicos.show()

In [ ]:
# Jogo com menor público
jogosIconicos = jogosIconicos.union(jogosCopa.filter(jogosCopa.Attendance == menorPublico))
jogosIconicos.show()

#inclui o dado no data freme por isto tem 2 dado

In [ ]:
# Jogos abaixo da média de público
jogosCopa.filter(jogosCopa.Attendance < mediaPublico).show(10)

In [ ]:
# Jogos acima da média de público
jogosCopa.filter(jogosCopa.Attendance >= mediaPublico).show(10)

In [ ]:
# Jogos por cidade
jogosCidade = jogosCopa.groupBy("City").count() \
                       .withColumnRenamed('City', 'Cidade') \
                       .withColumnRenamed('count', 'Num. jogos')

# Público total por cidade
publicoCidade = jogosCopa.groupBy("City").sum("Attendance") \
                         .withColumnRenamed('City', 'Cidade') \
                         .withColumnRenamed('sum(Attendance)', 'Público total')

# Análise do número de partidas e da média de público
jogosCidade.createOrReplaceTempView("tempJogosCidade")
publicoCidade.createOrReplaceTempView("tempPublicoCidade")
queryCidade = """
                  select a.Cidade, 
                         a.`Num. jogos`,
                         b.`Público total`,
                         round(b.`Público total` / a.`Num. jogos`) as `Média público por partida` 
                  from tempJogosCidade a
                  inner join tempPublicoCidade b
                  on a.cidade = b.cidade
                  order by a.`Num. jogos` desc
              """
cidadesJogos = spark.sql(queryCidade)

In [ ]:
# Amostra
cidadesJogos.show()

In [ ]:
# Exportar resultados
jogosIconicos.toPandas().to_csv('jogosIconicos.csv')
cidadesJogos.toPandas().to_csv('cidadeJogos.csv')

**Público nos jogos Brasil (*extra*)**
---


1.   Qual o ***maior*** público registrado em jogos do Brasil?
2.   Qual o ***menor*** público registrado em jogos do Brasil?
3.   Quais jogos do Brasil ficaram ***abaixo*** da média de público?
4.   Quais jogos do Brasil ficaram ***acima*** da média de público?
5.   Exporte os resultados em formato .csv, de acordo com:

      *   Itens 1 e 2 juntos
      *   Item 3 sozinho
      *   Item 4 sozinho






In [ ]:
# Maior e menor publico jogos Brasil 
Publico = jogosCopa.groupBy("Home Team Name","Away Team Name").max("Attendance") \
                         .withColumnRenamed('Home Team Name', 'Mandante') \
                         .withColumnRenamed('Away Team Name', 'Visitante') \
                         .withColumnRenamed('max(Attendance)', 'Público total')                  

Publico.createOrReplaceTempView("tempPublicoMandante")
querypublicobrasil = """
                            select 
                                  max(`Público total`) as Max_Publico_Jogos_Brazil,
                                  min(`Público total`) as Min_Publico_Jogos_Brazil
                            from tempPublicoMandante
                            where Mandante = 'Brazil'
                            OR Visitante = 'Brazil'
                        """
publicobrasil = spark.sql(querypublicobrasil)

publicobrasil.show()


In [ ]:
#Jogos do Brasil que ficaram abaixo da média de público
from pyspark.sql.functions import col
JogosBrasil = jogosCopa.filter(((col("Home Team Name") == "Brazil") | (col("Away Team Name") == "Brazil")))

AvgBrazil = JogosBrasil.agg({"Attendance": "avg"}).collect()[0][0]

JogosAbaixoMedia = JogosBrasil.filter(JogosBrasil.Attendance < AvgBrazil)
JogosAbaixoMedia.show(10)



In [ ]:
#Jogos do Brasil que ficaram acima da média de público
JogosBrasil = jogosCopa.filter(((col("Home Team Name") == "Brazil") | (col("Away Team Name") == "Brazil")))

AvgBrazil = JogosBrasil.agg({"Attendance": "avg"}).collect()[0][0]

JogosAcimaMedia = JogosBrasil.filter(JogosBrasil.Attendance > AvgBrazil)
JogosAcimaMedia.show(10)

In [ ]:
# Exportar resultados
publicobrasil.toPandas().to_csv('publicobrasil.csv')
JogosAbaixoMedia.toPandas().to_csv('JogosAbaixoMedia.csv')
JogosAcimaMedia.toPandas().to_csv('JogosAcimaMedia.csv')

# ***Resultados das partidas***

In [ ]:
# Resultados jogos
resultadosJogos = jogosCopa.select('Home Team Name', 
                                   'Away Team Name', 
                                   'Home Team Goals', 
                                   'Away Team Goals',
                                   'Home Team Initials',
                                   'Away Team Initials') \
                           .withColumnRenamed('Home Team Name', 'Mandante') \
                           .withColumnRenamed('Away Team Name', 'Visitante') \
                           .withColumnRenamed('Home Team Goals', 'Gols em casa') \
                           .withColumnRenamed('Away Team Goals', 'Gols visitante') \
                           .withColumnRenamed('Home Team Initials', 'Sigla Mandante') \
                           .withColumnRenamed('Away Team Initials', 'Sigla Visitante')

# Diferença de gols por jogo
queryDiffGols = """
                    select  Mandante,
                            Visitante, 
                            Gols_Casa, 
                            Gols_Fora, 
                            diff as Saldo
                      from 
                      ( 
                            select Mandante, 
                                   Visitante, 
                                  `Gols em casa` AS Gols_Casa, 
                                  `Gols visitante` AS Gols_Fora, 
                                   abs(`Gols em casa` - `Gols visitante`) as diff 
                            from tempResultados 
                      )
                      order by diff desc
                """

resultadosJogos.createOrReplaceTempView("tempResultados")
golsResultados=spark.sql(queryDiffGols)

In [ ]:
# Ordenado por maior saldo em ordem decrescente
golsResultados.show(10)

In [ ]:
# Consulta jogos Brasil (vitórias, derrotas e empates)
queryJogosBrasil = """
                        select resultado, 
                               count(*) as qtd
                        from 
                        (
                            select local, 
                                   case when diff > 0 then 'vitoria'
                                        when diff = 0 then 'empate'
                                        else 'derrota'
                                   end as resultado
                            from 
                            ( 
                                  select 'Mandante' as local, 
                                         `Gols em casa` - `Gols visitante` as diff 
                                  from tempResultados 
                                  where `Sigla Mandante` = 'BRA'
                            )

                            union all

                            select local, 
                                   case when diff < 0 then 'vitoria'
                                        when diff = 0 then 'empate'
                                        else 'derrota'
                                   end as resultado
                            from 
                            ( 
                                  select 'Visitante' as local, 
                                         `Gols em casa` - `Gols visitante` as diff 
                                  from tempResultados 
                                  where `Sigla Visitante` = 'BRA'
                            )
                        )
                        group by resultado
                   """

brasilResultados=spark.sql(queryJogosBrasil)

In [ ]:
# Jogos Brasil
brasilResultados.show()

In [ ]:
# Porcentagem de vitórias, derrotas e empates
vitorias = brasilResultados.filter(brasilResultados['resultado'] == 'vitoria').collect()[0][1]
derrotas = brasilResultados.filter(brasilResultados['resultado'] == 'derrota').collect()[0][1]
empates = brasilResultados.filter(brasilResultados['resultado'] == 'empate').collect()[0][1]
totalJogos = vitorias + derrotas + empates

print('Total de jogos: ' + str(totalJogos))
print('Vitorias: ' + str(round(100*(vitorias/totalJogos), 2)) + '%')
print('Derrotas: ' + str(round(100*(derrotas/totalJogos), 2)) + '%')
print('Empates: ' + str(round(100*(empates/totalJogos), 2)) + '%')

In [ ]:
# Adversários Brasil
queryAdversariosBrasil = """
                                select adversario,
                                       sum(qtd_partidas) as qtd_partidas
                                from 
                                (
                                    select `Sigla Visitante` as adversario,
                                           count(*) as qtd_partidas
                                    from tempResultados 
                                    where `Sigla Mandante` = 'BRA'
                                    group by `Sigla Visitante`

                                    union all

                                    select `Sigla Mandante` as adversario,
                                           count(*) as qtd_partidas
                                    from tempResultados 
                                    where `Sigla Visitante` = 'BRA'
                                    group by `Sigla Mandante`
                                )
                                group by adversario
                                order by qtd_partidas desc
                         """

brasilAdversarios=spark.sql(queryAdversariosBrasil)

In [ ]:
# Adversários Brasil em quantidade de partidas ordem decrescente
brasilAdversarios.show()

In [ ]:
# Exportar resultados
golsResultados.toPandas().to_csv('golsResultados.csv')
brasilAdversarios.toPandas().to_csv('brasilAdversarios.csv')

**Resultados partidas BRAxGER e histórico (*extra*)**
---


1.   Qual o ***empate*** com o maior número de gols na história das copas?
2.   Qual o ***número*** de derrotas, vitórias e empates em partidas do Brasil com a Alemanha?
3.   Qual a ***porcentagem*** de derrotas, vitórias e empates em partidas do Brasil com a Alemanha?
4.   Qual a ***maior derrota*** sofrida pelo Brasil?
5.   Qual a ***maior vitória*** do Brasil?
6.   Exportar os resultados no formato .csv, de acordo com: 
     *   Item 1 separadamente
     *   Itens 2 e 3 juntos em um único arquivo
     *   Itens 4 e 5 juntos em um único arquivo

In [ ]:
#Maior Empate
JogosEmpate = jogosCopa.filter(col('Home Team Goals') == col('Away Team Goals'))

Empate = JogosEmpate.agg({"Home Team Goals": "max"}).collect()[0][0]

JogosEmpate.filter(JogosEmpate['Home Team Goals'] == Empate).show()

In [ ]:
#Qual o número de derrotas, vitórias e empates em partidas do Brasil com a Alemanha?






# ***Seleção 1970***

In [ ]:
# Jogadores Ano 1970
jogosCopa.createOrReplaceTempView("tempJogos")
jogadoresCopa.createOrReplaceTempView("tempJogadores")

queryJogadores1970 = """
                          select  roundID,
                                  matchID,
                                  `Team Initials`,
                                  `Coach Name`,
                                  `Line-up`,
                                  `Shirt Number`,
                                  `Player Name`,
                                  position,
                                  event
                          from tempJogadores 
                          where matchID in
                          (
                              select matchID
                              from tempJogos
                              where year = 1970
                          )
                     """

jogadores1970=spark.sql(queryJogadores1970)

In [ ]:
# Amostra 
jogadores1970.show(10)

In [ ]:
# Seleção Brasil
jogadores1970.createOrReplaceTempView("tempJogadores1970")
queryJogadoresBrasil1970 = """
                               select distinct `Shirt Number` as `Num. Camisa`,
                                               `Player Name` as `Jogador` 
                               from tempJogadores1970
                               where `Team Initials` = 'BRA'
                               order by `Num. Camisa` asc
                           """

selecao1970=spark.sql(queryJogadoresBrasil1970)

In [ ]:
# Amostra
selecao1970.show()

In [ ]:
# Final 1970
idJogo = jogosCopa.filter((jogosCopa.Stage == 'Final') & (jogosCopa.Year == 1970)).collect()[0][17]

In [ ]:
# Eventos Final 1970
queryFinal1970 = """
                      select `Player Name` as Jogador,
                              event as Evento
                      from tempJogadores
                      where matchId = {0}
                      and {1} is not null
                 """.format(idJogo, 'event')
eventosFinal1970=spark.sql(queryFinal1970)

In [ ]:
print(queryFinal1970)

In [ ]:
# Amostra
eventosFinal1970.show()

In [ ]:
# Gols marcados ao longo da Copa 1970
queryGols1970 = """
                      select `Player Name` as Jogador,
                              sum(length(regexp_replace(event, '[^G]', ''))) as Gols
                      from tempJogadores
                      where matchID in
                      (
                        select matchID
                        from tempJogos
                        where year = 1970
                        and (`Home Team Initials` = 'BRA' or `Away Team Initials` = 'BRA')
                      )
                      and event like '%G%'
                      and `Team Initials` = 'BRA'
                      group by `Player Name`
                      order by Gols desc
                 """

golsBrasil1970=spark.sql(queryGols1970)

In [ ]:
# Amostra
golsBrasil1970.show()

In [ ]:
# Exportar resultados
selecao1970.toPandas().to_csv('selecao1970.csv')
golsBrasil1970.toPandas().to_csv('golsBrasil1970.csv')
eventosFinal1970.toPandas().to_csv('eventosFinal1970.csv')

**Extra**
---


1.   Fazer as mesmas análises para a seleção de **1998**
2.   Exportar, separadamente, os reultados no formato .csv

In [ ]:
#
# 
# ESCREVA AQUI!!
#
#